## pandas 문법 실습

In [51]:
# 예시 데이터프레임 생성

import numpy as np
import pandas as pd

# https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
# getting started - house prices data 예시로 사용

root_dir = '../data-examples'
df = pd.read_csv(root_dir + '/house-prices-advanced-regression-techniques/train.csv')
print(f'row 개수 : {len(df)}, column 개수 : {len(df.columns)}')

row 개수 : 1460, column 개수 : 81


### pandas display option

In [ ]:
# max_columns, max_rows 설정
pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_columns', None) # limit 없음
pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_rows', None) # limit 없음

In [ ]:
# 출력되는 item 수 limit 해제하기
pd.set_option('display.max_seq_items', None)

In [ ]:
# 출력창 크기 조절하기
pd.set_option('display.width', 10) # 한 줄에 하나씩 출력하고 싶을 때. 작은 수를 넣어서 사용

### data load method

In [ ]:
# csv load
filepath = root_dir+'/house-prices-advanced-regression-techniques/train.csv'
pd.read_csv(filepath)

In [ ]:
# 여러 형태 load
filepath = root_dir+'/house-prices-advanced-regression-techniques/train.csv'
pd.read_csv(filepath, sep='|', encoding='utf') # |로 구분되는 파일 & utf-8로 encoding된 파일

### null 처리 method

In [ ]:
# null 개수 파악
df.isna().sum() # column별로 나타냄
df.isna().sum(axis=1) # row별로 나타냄

In [ ]:
# null 행/열 제거
df.dropna(axis=0, inplace=True) # null 있는 row제거
df.dropna(axis=1, inplace=True) # null 있는 column제거

### 문자열 처리, 문자열 치환 정규 표현식

In [ ]:
# DataFrame 내의 문자열에 접근하는 방법
df.GarageQual[df.GarageQual.str[1]=='A'] # GarageQual의 string자료의 [1]이 'A'인 것만 추출. object자료형에는 안됨

In [ ]:
# 정규 표현식 이용해 문자열 일부 수정
df = df.replace(r'[^0-9A-Za-z가-힣]+','',regex=True) # (수정전, 수정후, regex=True) 정규표현식위해 regex=True
df['GarageQual'] = df['GarageQual'].str.replace('A', 'a') # 이처럼 정규표현식 없이도 부분 변경 가능하긴 함

### DataFrame 특정값에 접근해서 값 수정하기

In [ ]:
# 여러 개의 데이터에 접근하고 데이터 수정하기 (map 이용해서도 가능)
df['LotShape'][df['LotShape']=='Reg'] = '3' # LotShape 가 Reg인 행의, LotShape 열을 3으로 수정. 

In [ ]:
# 하나의 데이터 수정하기
col_id = list(df.columns).index('LandContour')
df.iloc[3, 8] = 10 # 3, 8 위치의 셀 값을 10으로 수정

### map(값 치환), applymap(DataFrame에 function 적용)

In [ ]:
# Series에서 map을 통한 치환
df['LotShape'] = df['LotShape'].map({'IR1':'IR', 'IR2':'IR', 'IR3':'IR', 'Reg':'Reg'})
df['LotShape'].head(10)

In [ ]:
# Series에서 map을 통한 func 적용
df['LotAreaClass'] = df['LotArea'].map(lambda x: 1 if x>10000 else 0)
df[['LotArea', 'LotAreaClass']].head(10)

In [ ]:
# DataFrame에서 applymap을 통한 func 일괄 적용 : 각 셀에 접근해서 계산함, 자주 사용 X
df[['MSSubClass','LotFrontage']].applymap(lambda cell : cell*20)

In [79]:
# DataFrame에서 apply를 통해 func 적용 : 각 행에 접근해서 계산함. axis=1 꼭 필요..
df['new_col'] = df[['MSSubClass','LotFrontage']].apply(lambda row:row['MSSubClass']*row['LotFrontage'], axis=1)

In [ ]:
# DataFrame에서 apply를 통해 func 적용 : 각 열에 접근하는 게 default
df[['MSSubClass','LotFrontage']].apply(sum)

### Groupby

#### 기준이 되는 범주형 변수의 각 범주마다, sum이나 mean, size, count 등 집계함수를 사용하는 방법

In [ ]:
# groupby 기본적 사용방법 : LotShape라는 열의 각 범주에 대해, OverallQual, MasVnrArea의 sum값을 구하는 것
df.groupby('LotShape', as_index=False).sum()[['LotShape', 'OverallQual', 'MasVnrArea']]
# as_index를 False로 하면 index를 포함하지 않게 됨, 그래서 집계함수 오른쪽에 열 선택할 때 Level에 사용된 col도 넣어줘야

In [ ]:
# 두 개의 기준에 대해 groupby하여 집계함수 짜기
df.groupby(['Street','LotShape']).sum()[['OverallQual','MasVnrArea']]
# as_index가 False이면 Street, LotShape가 index가 됨

In [ ]:
# 각 column에 대해 각기 다른 집계함수 적용하기
df.groupby(['Street','LotShape']).agg({'OverallQual':np.sum, 'MasVnrArea':np.mean})

In [ ]:
# 각 column에 대해 각기 다른 집계함수 적용하기 - 더 일반화된 방법
def func(x):
    return pd.Series([x.OverallQual.sum(), x.MasVnrArea.mean(), (x.OverallQual*x.MasVnrArea).mean()], 
                     index = ['OverallQualSUM','MasVnrAreaMEAN','MultiplyMEAN'])
# OverallQual에는 sum, MaxVnrArea에는 mean을 적용하고 또한 기존컬럼 두개 이상을 이용해 새 집계함수를 작성해서 적용
df.groupby(['Street','LotShape']).apply(func)

### MultiIndex(& MultiColumn) 처리 방법

In [ ]:
# Street, LotShape를 기준으로 한 groupby 결과
def func(x):
    return pd.Series([x.OverallQual.sum(), x.MasVnrArea.mean(), (x.OverallQual*x.MasVnrArea).mean()], 
                     index = ['OverallQualSUM','MasVnrAreaMEAN','MultiplyMEAN'])
groupby2 = df.groupby(['Street','LotShape']).apply(func)
groupby2

In [ ]:
# MultiIndex에서 Level 0(Street)의 Grvl에 접근
groupby2.loc['Grvl'] # 또는
groupby2.xs('Grvl', level = 'Street')
# MultiIndex에서 Level을 숫자로 표현
groupby2.xs('Grvl', level = 0)

In [ ]:
# MultiIndex에서 Level 1(LotShape)의 IR1에 접근 : 레벨 1 이상의 깊은 범주에는 loc을 쓸 수 없음
groupby2.xs('IR1', level = 'LotShape')
# MultiIndex에서 Level을 숫자로 표현
groupby2.xs('IR1', level = 1)

In [31]:
# MultiColumn 처리 : xs('label1', level = 0, axis = 1) : axis=1을 넣어주면 됨!!!!!!!

### DataFrame의 dtype 처리

In [ ]:
# 모든 컬럼의 dtype 확인
df.dtypes

In [ ]:
# dtype이 object인 column만 추출 (int64, float64등 다 됨)
df.dtypes[df.dtypes=='object'].index # Series로 리턴되기 때문에 columns말고 index에 접근해야함 

In [7]:
# dtype을 바꿔주는 방법 : YearBuilt, YearRemodAdd 열은 연도를 나타내는데 int64이므로 object로 바꿈
df['YearBuilt'] = df['YearBuilt'].astype('object')
df['YearRemodAdd'] = df['YearRemodAdd'].astype('object')

### 데이터프레임 병합 : concat

In [33]:
# 예시 DataFrame 생성
df1 = pd.DataFrame({'a':['a0','a1','a2','a3'],
                   'b':['b0','b1','b2','b3'],
                   'c':['c0','c1','c2','c3']},
                  index = [0,1,2,3])

df2 = pd.DataFrame({'a':['a2','a3','a4','a5'],
                   'b':['b2','b3','b4','b5'],
                   'c':['c2','c3','c4','c5'],
                   'd':['d2','d3','d4','d5']},
                   index = [2,3,4,5])

In [35]:
# DataFrame을 위아래로 붙이기 : axis=0
df_concat0 = pd.concat([df1, df2], axis=0, ignore_index = True) # ignore_index는 합친 dataframe에 새로운 index를 부여

In [38]:
# DataFrame을 왼쪽오른쪽으로 붙이기 : axis=1
df_concat1 = pd.concat([df1, df2], axis=1, ignore_index = True) # 마찬가지로 index를 새로 부여

In [ ]:
# 중요 : 가로로 붙이든, 세로로 붙이든 공통되는 열/행 이름을 기준으로 붙여진다.
# 같은 종류의 데이터인데 df1, df2에서 열의 이름이 다르다면 제대로 붙여지지 않음
# axis=1 병합에서 4개의 index가 순서대로 붙게 하는 방법 : index initialization 먼저 해주기. (axis=0에선 column 초기화)
df1.index = np.arange(4)
df2.index = np.arange(4)
pd.concat([df1, df2], axis=1, ignore_index = True)

In [ ]:
# outer와 inner 옵션 : default는 outer. 한쪽 df에 없는 컬럼은 Nan값으로 채움
pd.concat([df1, df2], axis=0)

In [ ]:
# inner : 한쪽 df에 없는 컬럼은 버린다.
pd.concat([df1, df2], axis=0, join='inner')

### 데이터프레임 병합 : merge (수정필요)

- concat은 물리적으로 단순히 이어붙이는 작업
- merge는 공통된 열을 기준으로 df와 df를 연결하는 작업

북마크에 있는데 샘플데이터가 없어서 자세한건 북마크를 참조합시다